# 빠른 시작

이 빠른 시작은 코드를 직접 살펴보고 🤗 SetFit 모델을 훈련하고 사용하는 방법의 예를 보려는 개발자를 위한 것입니다. 이 빠른 시작으로 시작한 다음 추가 자료를 보려면 [튜토리얼](https://huggingface.co/docs/setfit/main/en/./tutorials/overview) 또는 [방법 가이드](https://huggingface.co/docs/setfit/main/en/./how_to/overview)로 진행하는 것이 좋습니다. 또한 [개념 가이드](https://huggingface.co/docs/setfit/main/en/./conceptual_guides/setfit)는 SetFit이 정확히 어떻게 작동하는지 설명하는 데 도움이 됩니다.

🤗 SetFit을 설치하는 것으로 시작합니다.

```bash
pip install setfit
```

CUDA 지원 그래픽 카드가 있는 경우 훨씬 더 빠르게 훈련하고 추론을 수행하려면 [CUDA 지원으로 `torch` 설치](https://pytorch.org/get-started/locally/)하는 것이 좋습니다.

```bash
pip install torch --index-url https://download.pytorch.org/whl/cu118
```

## SetFit

SetFit은 적은 훈련 데이터로 짧은 대기 시간의 텍스트 분류 모델을 훈련하기 위한 효율적인 프레임워크입니다. 이 빠른 시작에서는 SetFit 모델을 훈련하는 방법, 추론을 수행하는 방법, Hugging Face Hub에 저장하는 방법을 배웁니다.

### 훈련

이 섹션에서는 [Sentence Transformer 모델](https://huggingface.co/models?library=sentence-transformers)을 로드하고 영화 리뷰를 긍정 또는 부정으로 분류하도록 추가 미세 조정합니다. 모델을 훈련하려면 다음 세 가지를 준비해야 합니다. 1) **모델**, 2) **데이터 세트**, 3) **훈련 인수**.

**1**. 선택한 Sentence Transformer 모델을 사용하여 SetFit 모델을 초기화합니다. 어떤 Sentence Transformer 모델을 선택할지 결정하는 데 [MTEB 리더보드](https://huggingface.co/spaces/mteb/leaderboard)를 사용하는 것을 고려하십시오. 작지만 성능이 좋은 모델인 [BAAI/bge-small-en-v1.5](https://huggingface.co/BAAI/bge-small-en-v1.5)를 사용합니다.

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")

**2a**. 다음으로 [SetFit/sst2](https://huggingface.co/datasets/sst2) 데이터 세트의 "train" 및 "test" 분할을 모두 로드합니다. 데이터 세트에는 `"text"` 및 `"label"` 열이 있습니다. 이것이 바로 🤗 SetFit이 예상하는 형식입니다. 데이터 세트에 다른 열이 있는 경우 4단계에서 [Trainer](https://huggingface.co/docs/setfit/main/en/reference/trainer#setfit.Trainer)의 column_mapping 인수를 사용하여 열 이름을 `"text"` 및 `"label"`에 매핑할 수 있습니다.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("SetFit/sst2")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 6920
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 1821
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 872
    })
})

**2b**. 실제 시나리오에서는 약 7,000개의 고품질 레이블이 지정된 훈련 샘플을 갖는 것이 매우 드물기 때문에 실제 환경에서 🤗 SetFit이 어떻게 작동하는지 더 잘 파악하기 위해 훈련 데이터 세트를 크게 줄입니다. 구체적으로 `sample_dataset` 함수는 각 클래스에 대해 8개의 샘플만 샘플링합니다. 테스트 세트는 더 나은 평가를 위해 영향을 받지 않습니다.

In [ ]:
from setfit import sample_dataset

train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=8)
train_dataset

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 16
})

In [ ]:
test_dataset = dataset["test"]
test_dataset

Dataset({
    features: ['text', 'label', 'label_text'],
    num_rows: 1821
})

**2c**. 데이터 세트의 레이블을 모델에 적용하여 예측이 읽을 수 있는 클래스를 출력하도록 할 수 있습니다. `SetFitModel.from_pretrained()`에 직접 레이블을 제공할 수도 있습니다.

In [ ]:
model.labels = ["negative", "positive"]

**3**. 훈련을 위한 [TrainingArguments](https://huggingface.co/docs/setfit/main/en/reference/trainer#setfit.TrainingArguments)를 준비합니다. 🤗 SetFit으로 훈련하는 것은 내부적으로 **임베딩 미세 조정**과 **분류 헤드 훈련**의 두 단계로 구성됩니다. 결과적으로 일부 훈련 인수는 튜플일 수 있으며, 두 값은 각각 두 단계에 사용됩니다.

`num_epochs` 및 `max_steps` 인수는 총 훈련 단계 수를 늘리거나 줄이는 데 자주 사용됩니다. SetFit을 사용하면 **더 많은 훈련이 아니라 더 많은 데이터**로 더 나은 성능을 얻을 수 있다는 점을 고려하십시오! 데이터가 많은 경우 1 에포크 미만으로 훈련하는 것을 두려워하지 마십시오.

In [ ]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=32,
    num_epochs=10,
)

**4**. [Trainer](https://huggingface.co/docs/setfit/main/en/reference/trainer#setfit.Trainer)를 초기화하고 훈련을 수행합니다.

In [ ]:
from setfit import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)

In [ ]:
trainer.train()

***** 훈련 실행 중 *****
  예제 수 = 5
  에포크 수 = 10
  총 최적화 단계 = 50
  총 훈련 배치 크기 = 32
{'embedding_loss': 0.2077, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.2}                                                                                                                
{'embedding_loss': 0.0097, 'learning_rate': 0.0, 'epoch': 10.0}                                                                                                                                 
{'train_runtime': 14.705, 'train_samples_per_second': 108.807, 'train_steps_per_second': 3.4, 'epoch': 10.0}
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:08<00:00,  5.70it/s]

**5**. 제공된 테스트 데이터 세트를 사용하여 평가를 수행합니다.

In [ ]:
trainer.evaluate(test_dataset)

***** 평가 실행 중 *****
{'accuracy': 0.8511806699615596}

클래스당 샘플 수를 늘려 정확도 향상을 관찰해 보십시오. 도전 과제로 클래스당 샘플 수, 학습률, 에포크 수, 최대 단계 수, 기본 Sentence Transformer 모델을 조정하여 매우 적은 데이터로 정확도를 90% 이상으로 향상시켜 보십시오.

### 🤗 SetFit 모델 저장

훈련 후 🤗 SetFit 모델을 로컬 파일 시스템이나 Hugging Face Hub에 저장할 수 있습니다. `save_directory`를 제공하여 `SetFitModel.save_pretrained()`를 사용하여 모델을 로컬 디렉터리에 저장합니다.

In [ ]:
model.save_pretrained("setfit-bge-small-v1.5-sst2-8-shot")

또는 `repo_id`를 제공하여 `SetFitModel.push_to_hub()`를 사용하여 모델을 Hugging Face Hub에 푸시합니다.

In [ ]:
model.push_to_hub("tomaarsen/setfit-bge-small-v1.5-sst2-8-shot")

### 🤗 SetFit 모델 로드

🤗 SetFit 모델은 1) Hugging Face Hub의 `repo_id` 또는 2) 로컬 디렉터리 경로를 제공하여 `SetFitModel.from_pretrained()`를 사용하여 로드할 수 있습니다.

In [ ]:
model = SetFitModel.from_pretrained("tomaarsen/setfit-bge-small-v1.5-sst2-8-shot") # Hugging Face Hub에서 로드

model = SetFitModel.from_pretrained("setfit-bge-small-v1.5-sst2-8-shot") # 로컬 디렉터리에서 로드

### 추론

🤗 SetFit 모델이 훈련되면 [SetFitModel.predict()](https://huggingface.co/docs/setfit/main/en/reference/main#setfit.SetFitModel.predict) 또는 [SetFitModel.__call__()](https://huggingface.co/docs/setfit/main/en/reference/main#setfit.SetFitModel.__call__)를 사용하여 리뷰를 분류하는 추론에 사용할 수 있습니다.

In [ ]:
preds = model.predict([
    "매력적이고 종종 감동적인 여정입니다.",
    "매우, 매우 느립니다.",
    "때로는 지루한 영화입니다.",
])
preds

['positive' 'negative' 'negative']

이러한 예측은 `model.labels`에 의존합니다. 설정되지 않은 경우 훈련 중에 사용된 형식(예: `tensor([1, 0, 0])`)으로 예측을 반환합니다.

## 다음 단계는 무엇인가요?

🤗 SetFit 빠른 시작을 완료했습니다! 🤗 SetFit 모델로 훈련, 저장, 로드 및 추론을 수행할 수 있습니다!

다음 단계로 [방법 가이드](https://huggingface.co/docs/setfit/main/en/./how_to/overview)를 살펴보고 하이퍼파라미터 검색, 지식 증류 또는 제로샷 텍스트 분류와 같은 특정 작업을 수행하는 방법을 알아보십시오. 🤗 SetFit이 어떻게 작동하는지 자세히 알아보려면 커피 한 잔을 들고 [개념 가이드](https://huggingface.co/docs/setfit/main/en/./conceptual_guides/setfit)를 읽어보십시오!

## 종단 간

이 스니펫은 전체 빠른 시작을 종단 간 예제로 보여줍니다.

In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import load_dataset

# 새로운 SetFit 모델 초기화
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", labels=["negative", "positive"])

# 데이터셋 준비
dataset = load_dataset("SetFit/sst2")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=8)
test_dataset = dataset["test"]

# 훈련 인수 준비
args = TrainingArguments(
    batch_size=32,
    num_epochs=10,
)

# 트레이너 준비
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)
trainer.train()

# 평가
metrics = trainer.evaluate(test_dataset)
print(metrics)
# => {'accuracy': 0.8511806699615596}

# 훈련된 모델 저장
model.save_pretrained("setfit-bge-small-v1.5-sst2-8-shot")
# 또는
model.push_to_hub("tomaarsen/setfit-bge-small-v1.5-sst2-8-shot")

# 훈련된 모델 로드
model = SetFitModel.from_pretrained("tomaarsen/setfit-bge-small-v1.5-sst2-8-shot") # Hugging Face Hub에서 로드
# 또는
model = SetFitModel.from_pretrained("setfit-bge-small-v1.5-sst2-8-shot") # 로컬 디렉터리에서 로드

# 추론 수행
preds = model.predict([
    "매력적이고 종종 감동적인 여정입니다.",
    "매우, 매우 느립니다.",
    "때로는 지루한 영화입니다.",
])
print(preds)
# => ["positive", "negative", "negative"]